In [2]:
from apify_client import ApifyClient
import pandas as pd  # Import pandas for DataFrame manipulation

# You can find your API token at https://console.apify.com/settings/integrations.
TOKEN = 'apify_api_K6W2UU0OLkFiIGipqblSCiJQVaMakt2nBOKh'

In [23]:
def hashtags_actor(hashtags: list, results_type: str, results_limit: int) -> dict:
    client = ApifyClient(TOKEN)
    # Prepare the Actor input
    run_input = {
        "hashtags": hashtags,
        "resultsType": results_type,
        "resultsLimit": results_limit,
    }
    # Run the Actor and wait for it to finish
    run = client.actor("reGe1ST3OBgYZSsZJ").call(run_input=run_input)
    data = []  # Initialize an empty list to store the results
    # Fetch Actor results from the run's dataset
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        data.append(item)
    # Create a DataFrame from the fetched data
    df = pd.DataFrame(data)
    return df

def hashtags_df() -> None:
    hashtags = ['foodiesmadrid', 'restaurantesmadrid']
    results_type = 'stories' # 'posts', 'stories'
    results = 1
    df = hashtags_actor(hashtags, results_type, results)
    # Extract the 'ownerUsername' column and convert it to a list
    if 'ownerUsername' in df.columns:
        owner_usernames = df['ownerUsername'].tolist()
        print("Owner Usernames:", owner_usernames)
    else:
        print("The 'ownerUsername' column is not present in the dataset.")
    return df, owner_usernames    

In [ ]:
df1 = df[['ownerUsername', 'likesCount', 'commentsCount', 'isSponsored', 'timestamp', 'locationName', 'caption','hashtags']].copy()
df1['timestamp'] = pd.to_datetime(df['timestamp']).dt.date  # Convert timestamp to datetime and extract the date
df1.rename({'timestamp':'date'}, axis='columns')

# Profile Analisis from a Profile Standard

Our goal is to amplify our share of IG and TikTok influencers to which our main concern is finding profiles relevant enough for getting leads and contacts.

Since we have a fair share of profiles already marked down, we'll use those to understand the use of Hashtags and Biography keywords, with a probable use of Caption use for later analysis.


In [3]:
def users_actor(direct_urls: list, results_limit: int) -> dict:
    client = ApifyClient(TOKEN)
    # Prepare the Actor input
    run_input = {
        "addParentData": False,
        "directUrls": direct_urls,
        "enhanceUserSearchWithFacebookPage": False,
        "isUserReelFeedURL": False,
        "isUserTaggedFeedURL": False,
        "resultsLimit": results_limit,
        "resultsType": "details",
        "searchLimit": 1,
        "searchType": "hashtag"
        }
    # Run the Actor and wait for it to finish
    run = client.actor("shu8hvrXbJbY3Eb9W").call(run_input=run_input)
    data = []  # Initialize an empty list to store the results
    # Fetch Actor results from the run's dataset
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        data.append(item)
    # Create a DataFrame from the fetched data
    df_actors = pd.DataFrame(data)
    return df_actors

def users_df(usernames: list) -> None:
    direct_urls = [
        "https://www.instagram.com/" + username + "/"
        for username in usernames
    ]
    results_limit = 200
    df_users = users_actor(direct_urls, results_limit)
    df_users = df_users[['username', 'biography', 'followersCount', 'followsCount', 'highlightReelCount', 'relatedProfiles', 'postsCount', 'latestPosts']]
    df_latestposts = df_users.explode('latestPosts')
    df_latestposts = df_latestposts['latestPosts'].apply(pd.Series)
    return df_users, df_latestposts


In [ ]:
standardUsernames = [
    "nievesmy",
    "lossibareats",
    "tipifood",
    "mariamartinsan_",
    "micaelaa.eats",
    "diegopradam",
    "alwayseatingmad",
    "ieat_out",
    "gastroandcook",
    "bocadictosss",
    "trucooplato",
    "aviles_av",
    "gourmitas_foodie",
    "crisbarberomad",
    "_gloriamj",
    "momento.madrid",
]
dfUserStandard, dfLatestPostsStandard = users_df(usernames = standardUsernames)


<>:21: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:21: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
C:\Users\samue\AppData\Local\Temp\ipykernel_14144\4162461407.py:21: SyntaxWarning: invalid escape sequence '\d'
  dfUserStandard.to_csv(save_path + "\dfUserStandard.csv", index=False)
C:\Users\samue\AppData\Local\Temp\ipykernel_14144\4162461407.py:22: SyntaxWarning: invalid escape sequence '\d'
  dfLatestPostsStandard.to_csv(save_path + "\dfLatestPostsStandard.csv", index=False)


In [9]:
save_path = "C:/Users/samue/Documents"
dfUserStandard.to_csv(save_path + "/dfUserStandard.csv", index=False, sep=';', encoding="utf-8", errors='ignore')
dfLatestPostsStandard.to_csv(save_path + "/dfLatestPostsStandard.csv", index=False, sep=';', encoding="utf-8", errors='ignore')

In [ ]:
print(dfUserStandard)  # Display the first few rows of the DataFrame

                                       inputUrl           id  \
0           https://www.instagram.com/tipifood/  41926395892   
1        https://www.instagram.com/diegopradam/     11833008   
2      https://www.instagram.com/micaelaa.eats/  41779842024   
3           https://www.instagram.com/ieat_out/  62239231676   
4    https://www.instagram.com/mariamartinsan_/    449512654   
5    https://www.instagram.com/alwayseatingmad/  57774345717   
6           https://www.instagram.com/nievesmy/    189850254   
7          https://www.instagram.com/_gloriamj/     30082189   
8          https://www.instagram.com/aviles_av/  68057036233   
9      https://www.instagram.com/gastroandcook/  64419248593   
10      https://www.instagram.com/bocadictosss/  39040154195   
11       https://www.instagram.com/trucooplato/  44498263218   
12    https://www.instagram.com/momento.madrid/  36573825003   
13      https://www.instagram.com/lossibareats/  51248611248   
14  https://www.instagram.com/gourmitas_

In [ ]:
from collections import Counter
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download('punkt_tab')

stop_words = set(stopwords.words("english") + stopwords.words("spanish"))


{'hasn', 'en', 'estemos', 'sentidos', 'tuviéramos', 'during', "you've", 'eras', 'esos', "mustn't", 'estuviéramos', 'nos', 'habría', 'hubierais', 'sentido', 'mi', 'esas', 'hayan', 'tenida', 'before', 'she', 'estarán', 'otro', 'por', 'have', 'éramos', 'fueran', 'be', 'habiendo', 'estando', 'of', 'tengáis', 'but', "shan't", "i've", 'mustn', 'de', 'estuvieseis', 'tuviésemos', 'haya', 'won', 'shan', 'then', 'them', 'tuvieses', 'algunos', 'estáis', 'nosotros', 'vuestro', 'hubieseis', 'over', 'tuyo', 'fuese', 'm', 'algo', "shouldn't", 'o', 'estuviste', 'are', 'hubiese', 'tienen', 'os', 'our', 'ante', "it'd", 'being', 'mucho', 'theirs', 'you', 'ya', 'needn', 'too', 'el', 'estada', 'he', 'tenidas', 'tuvisteis', 'tuvimos', 'tuvierais', 'vuestra', 'estamos', 'habido', 'ti', 'las', 'mis', 'tuviese', 'lo', 'porque', 'tuyas', 'habíamos', 'ellos', 'tuviera', 'mío', 'if', 'sean', 'tuvo', 'his', 'estés', 'tuviste', 'durante', 'algunas', 'eran', 'seríais', 'having', 'tenga', 'myself', 'until', 'tenemos'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [1]:
def extractStandardFeatures(profiles_df, posts_df, top_n=10):
    # Combine all captions and hashtags
    all_bios = " ".join(profiles_df["biography"])
    all_hashtags = ' '.join(posts_df['hashtags'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x) if pd.notna(x) else '')
    )
    
    # Clean text (remove stopwords, punctuation)
    bios_words = nltk.word_tokenize(all_bios.lower())
    bios_words = [w for w in bios_words if w.isalpha() and w not in stop_words]
    hashtags_words = nltk.word_tokenize(all_hashtags)
    hashtags_words = [w for w in hashtags_words if w.isalpha() and w not in stop_words] 

    # Top keywords and hashtags
    top_bios = Counter(bios_words).most_common(top_n)
    top_hashtags = Counter(hashtags_words).most_common(top_n)

    top_bios = [kw[0] for kw in top_bios]
    top_hashtags = [tag[0] for tag in top_hashtags]

    return top_bios, top_hashtags

In [ ]:

top_bios, top_hashtags = extractStandardFeatures(profiles_df = dfUserStandard, posts_df = dfLatestPostsStandard, top_n=15)
# Define elements to remove
hashtags_to_remove = ['gourmitasfoodie', 'LOSSIBAREATS']
bios_to_remove = ['nathalie', 'andorra', 'aquí']

# Filter the lists
top_hashtags = [tag for tag in top_hashtags if tag not in hashtags_to_remove]
top_bios = [bio for bio in top_bios if bio not in bios_to_remove]

Standard Hashtags: ['madrid', 'restaurantesmadrid', 'planesmadrid', 'foodie', 'madridfood', 'comerenmadrid', 'food', 'foodiesmadrid', 'foodiemadrid', 'planesenmadrid', 'madridfoodies', 'foodmadrid', 'gourmitasfoodie', 'LOSSIBAREATS', 'madridfoodie']
Standard Bios: ['restaurantes', 'madrid', 'planes', 'enseño', 'confianza', 'nuevos', 'experiencias', 'cuenta', 'recetas', 'aquí', 'foodies', 'andorra', 'inessafrica', 'mallorca', 'acompáñame']
<class 'list'>


In [79]:
print("Filtered Hashtags:", top_hashtags)
print("Filtered Bios:", top_bios)

Filtered Hashtags: ['madrid', 'restaurantesmadrid', 'planesmadrid', 'foodie', 'madridfood', 'comerenmadrid', 'food', 'foodiesmadrid', 'foodiemadrid', 'planesenmadrid', 'madridfoodies', 'foodmadrid', 'madridfoodie']
Filtered Bios: ['restaurantes', 'madrid', 'planes', 'enseño', 'confianza', 'nuevos', 'experiencias', 'cuenta', 'recetas', 'foodies', 'inessafrica', 'mallorca', 'acompáñame']


In [ ]:
def compare_to_standard(profile, standard_features):
    # Extract profile's hashtags and captions
    profileToCompare = ['fancyfree_lis']
    df_profileToCompare, _ = users_df(usernames = profileToCompare)
    

    profile_hashtags = set(tag for post in profile["posts"] for tag in post.get("hashtags", []))
    profile_captions = " ".join(post["caption"] for post in profile["posts"] if post.get("caption"))
    
    # Hashtag similarity (Jaccard)
    standard_hashtags = set(standard_features["hashtags"])
    hashtag_similarity = len(profile_hashtags & standard_hashtags) / len(profile_hashtags | standard_hashtags) if profile_hashtags else 0
    
    # Caption similarity (TF-IDF)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([profile_captions, " ".join(standard_features["keywords"])])
    caption_similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    
    return {
        "username": profile["username"],
        "hashtag_similarity": hashtag_similarity,
        "caption_similarity": caption_similarity,
        "overall_score": 0.6 * caption_similarity + 0.4 * hashtag_similarity,  # Weighted
    }

# Compare all scraped profiles
similarity_results = []
for _, profile in profiles_df.iterrows():
    similarity_results.append(compare_to_standard(profile, standard_features))

results_df = pd.DataFrame(similarity_results).sort_values("overall_score", ascending=False)
print(results_df.head())